# VacationPy
----

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# Create city weather data frame
filepath = '/Users/kiranrangaraj/Desktop/Homework/python-api-challenge/output_data/cities.csv'
city_data_df = pd.read_csv(filepath)
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vaini,-21.20,-175.20,77.00,61,20,10.29,TO,1595292455
1,1,punta arenas,-53.15,-70.92,30.20,92,8,6.93,CL,1595292251
2,2,souillac,-20.52,57.52,68.00,68,75,11.41,MU,1595292473
3,3,atuona,-9.80,-139.03,80.24,80,94,18.07,PF,1595292458
4,4,lesozavodsk,45.48,133.42,71.98,77,100,6.44,RU,1595292918


In [5]:
# Identify maximum humidity in data frame
city_data_df['Humidity'].max()

100

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [7]:
# Store latitude and longitude as locations and use humidity for weight
locations = city_data_df[['Lat', 'Lng']]
weight = city_data_df['Humidity']

# Plot heatmap
fig = gmaps.figure()

# Add a heat layer to the map
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100, point_radius=3)

# Add the heat layer
fig.add_layer(heat_layer)

# Show the heat map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Define ideal weather conditions and isolate
ideal_vacation_weather = city_data_df.loc[(city_data_df['Max Temp'] < 78) & (city_data_df['Max Temp'] > 65) \
                                         & (city_data_df['Humidity'] < 50) \
                                         & (city_data_df['Cloudiness'] < 30) \
                                         & (city_data_df['Wind Speed'] < 15)]
# Drop any rows with null values
ideal_vacation_weather = ideal_vacation_weather.dropna()
ideal_vacation_weather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,omsukchan,62.53,155.80,68.63,39,0,5.88,RU,1595292921
35,35,komsomolskiy,40.43,71.72,73.40,40,0,9.17,UZ,1595292923
107,107,kodiak,57.79,-152.41,75.20,44,1,13.87,US,1595292891
180,180,eau claire,44.81,-91.50,77.00,47,1,2.86,US,1595292937
215,215,seymchan,62.88,152.43,76.06,38,1,9.71,RU,1595292954


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Store ideal weather data frame into a new data frame named 'hotel_df'
hotel_df = ideal_vacation_weather[['City', 'Country', 'Lat', 'Lng']].copy()
# Add a 'Hotel Name' column to the new data frame
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
20,omsukchan,RU,62.53,155.80,
35,komsomolskiy,UZ,40.43,71.72,
107,kodiak,US,57.79,-152.41,
180,eau claire,US,44.81,-91.50,
215,seymchan,RU,62.88,152.43,


In [10]:
# Set parameters to search for a hotel
params = {
    'radius': 5000,
    'types': 'lodging',
    'key': g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, 'Hotel Name'] = name_address['results'][0]['name']
    except (KeyError, IndexError):
        print('Missing result, skipping to next.')

hotel_df

Missing result, skipping to next.
Missing result, skipping to next.
Missing result, skipping to next.


,City,Country,Lat,Lng,Hotel Name
20,omsukchan,RU,62.53,155.80,Zori
35,komsomolskiy,UZ,40.43,71.72,Ikathouse
107,kodiak,US,57.79,-152.41,Best Western Kodiak Inn And Convention Center
180,eau claire,US,44.81,-91.50,Hampton Inn Eau Claire
215,seymchan,RU,62.88,152.43,
236,cochrane,CA,51.18,-114.47,Days Inn & Suites by Wyndham Cochrane
323,zavetnoye,RU,47.12,43.89,
402,chulman,RU,56.85,124.91,Gostinitsa
423,lodja,CD,-3.48,23.43,
435,porangatu,BR,-13.44,-49.15,Wembley Park Hotel


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))